In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed, dump
from itertools import product
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import butter, filtfilt
import os
import gc
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [1]:
# load data
def load_data(filename):
    # load dataset
    exames_dict = loadmat(filename)
    
    target = exames_dict['target']
    exames = exames_dict['exames']

    # samples as the first dimension
    exames = np.transpose(exames,(2,0,1))

    target = target[0]
    return exames, target

def filt_data(data, order, fs, f1, f2, band):
    nyq = 0.5*fs  # nyquist frequency

    a1,b1 = butter(order,[(f1-(band/2))/nyq,(f1+(band/2))/nyq],btype='bandpass')
    a2,b2 = butter(order,[(f2-(band/2))/nyq,(f2+(band/2))/nyq],btype='bandpass')

    dataf1 = filtfilt(a1,b1,data,axis=1)
    dataf2 = filtfilt(a2,b2,data,axis=1)

    return dataf1, dataf2

def get_energy(data):
    data = np.square(data)
    return np.sum(data,axis=1)

In [2]:
def create_model(neurons1=1, neurons2=0, n_features=None):
    # create model
    model = Sequential()
    model.add(Dense(neurons1, input_dim=n_features, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    if neurons2 != 0:
        model.add(Dense(neurons2, kernel_initializer='uniform', activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# Analysis function for parallel execution
def cv_and_fit(params, train_index, test_index):

    # mean zero and unit variance
    scaler = StandardScaler()
    train_data = scaler.fit_transform(features[train_index, :])
    test_data = scaler.transform(features[test_index, :])
    del scaler

    # PCA
    pca = PCA(n_components=params['npcs'])
    train_data = pca.fit_transform(train_data)
    test_data = pca.transform(test_data)
    del pca

    # normalization to [-1 1] range
    scaler = MaxAbsScaler()
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)
    del scaler

    # Multiple initiations maintaining the best result
    best_acc = 0
    #best_history = None
    #best_model = None

    for init in range(n_inits):

        model = create_model(params['neurons1'], params['neurons2'], params['npcs'])
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

        history = model.fit(train_data, target[train_index], batch_size=25,
                            validation_data=(test_data, target[test_index]),
                            shuffle=True, epochs=2000, verbose=0, callbacks=[es])

        # score of loss and accuracy for the model trained
        score, acc = model.evaluate(test_data, target[test_index], batch_size=12, verbose=0)

        if acc > best_acc:
            best_acc = acc
            #best_history = history
            #best_model = model

        del history
        del model
        gc.collect()

    results_partial = {}
    results_partial['npcs'] = params['npcs']
    results_partial['neurons1'] = params['neurons1']
    results_partial['neurons2'] = params['neurons2']
    results_partial['acc'] = best_acc
    #results_partial['model'] = best_model
    #results_partial['history'] = best_history


    #del best_history
    #del best_model
    gc.collect()

    return results_partial

In [20]:
fs = 601.5  # sampling rate
f1 = 31.13  # modulating frequency 1
f2 = 39.36  # modulating frequency 2

band = 0.4
order = 4

win_size = 64
nwin = 29

n_jobs = -3
n_folds = 10    # number of folds for k-fold
n_inits = 10    # number of inits for each fold of k-fold

neurons1 = list(range(11, 21))  # neurons 1st hidden layer
neurons2 = list(range(0, 11))  # neurons 2nd hidden layer
npcs = [16]  # num pcs to keep

# grid with the parameters values
param_grid = ParameterGrid({'npcs': npcs,
                            'neurons1': neurons1,
                            'neurons2': neurons2})


# Start Analysis
exames, target = load_data('/home/pedrosergiot/Documents/Exames2_sem1segundo.mat')

dataf1, dataf2 = filt_data(exames[:,0:29,:], order, fs, f1, f2, band)

energyf1 = get_energy(dataf1)
energyf2 = get_energy(dataf2)

# concatenates the energy values obtained
features = np.concatenate((energyf1, energyf2), axis=1)

# CV K-Fold
kf = KFold(n_splits=n_folds)

# Executing analysis in parallel
parallel = Parallel(n_jobs=n_jobs, verbose=1, backend='loky')
out = parallel(delayed(cv_and_fit)(params, train_index, test_index) 
               for params, (train_index, test_index) in product(param_grid, kf.split(features)))


# Saving results for a given number of windows
file_name = "Test_best_parameters"
dump(out, file_name)

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-3)]: Done 188 tasks      | elapsed: 22.7min
[Parallel(n_jobs=-3)]: Done 438 tasks      | elapsed: 58.0min
[Parallel(n_jobs=-3)]: Done 788 tasks      | elapsed: 105.9min
[Parallel(n_jobs=-3)]: Done 1100 out of 1100 | elapsed: 145.4min finished


['Test_best_parameters']

In [24]:
results_df = pd.DataFrame(out)
    
gf = results_df.groupby(['npcs','neurons1','neurons2'])    
max_mean1 = 0
    
for key in gf.groups.keys():        
    if gf.get_group(key)['acc'].mean() > max_mean1:
        max_mean1 = gf.get_group(key)['acc'].mean()
        max_std1 = gf.get_group(key)['acc'].std()
        max_key1 = key
    
print([max_mean1, max_std1, max_key1])

[0.6321428537368774, 0.09008040939459754, (16, 19, 5)]
